In [1]:
%load_ext autoreload
import glob
import nibabel
import pandas as pd
import numpy as np
import os
import sys
from collections import defaultdict, Counter
from matplotlib import pyplot as plt
import matplotlib
from tqdm import tqdm
from joblib import Parallel, delayed

from IPython.core.debugger import set_trace

import torch
from torch import nn
from torch.nn import functional as F
from utils import create_dicts, load, video, trim

from multiprocessing import cpu_count
N_CPU = cpu_count()

from IPython.core.display import display, HTML
from celluloid import Camera

SEED = 42
%autoreload 2

In [2]:
# labels_components = np.load('labels_info.npy', allow_pickle=True).item()
# single_component_keys = {k for k,v in labels_components.items() if len(v['cc3d'][0]) == 2}

In [3]:
# USE_GEOM_FEATURES = False

# root_label = '../fcd_data/normalized_label'
# root_data = '../fcd_data/normalized_data/'

# paths_dict = create_dicts(root_label,
#                          root_data,
#                          None, 
#                          None,
#                          USE_GEOM_FEATURES, 
#                          None)

In [4]:
# root = '../fcd_data/normalized_tensors'
# paths = [os.path.join(root, f'{k}') for k in os.listdir(root)]

In [5]:
root_labels = '../fcd_data/data_only_usefull_areas/full/labels/'
root_brains = '../fcd_data/data_only_usefull_areas/full/t1_brains/'

paths_labels = [os.path.join(root_labels, f'{k}') for k in os.listdir(root_labels)]
paths_brains = [os.path.join(root_brains, f'{k}') for k in os.listdir(root_brains)]

In [6]:
for i,p in enumerate(paths_brains):
    if '9' in p:
        print(i,p )

7 ../fcd_data/data_only_usefull_areas/full/t1_brains/G294.npy
16 ../fcd_data/data_only_usefull_areas/full/t1_brains/49.npy
19 ../fcd_data/data_only_usefull_areas/full/t1_brains/39.npy
21 ../fcd_data/data_only_usefull_areas/full/t1_brains/n9.npy
39 ../fcd_data/data_only_usefull_areas/full/t1_brains/G409.npy
51 ../fcd_data/data_only_usefull_areas/full/t1_brains/n19.npy
57 ../fcd_data/data_only_usefull_areas/full/t1_brains/19.npy
68 ../fcd_data/data_only_usefull_areas/full/t1_brains/9.npy
72 ../fcd_data/data_only_usefull_areas/full/t1_brains/n29.npy


In [27]:
indx = 68
brain = np.load(paths_brains[indx])
label = np.load(paths_labels[indx])

plt.ioff()
camera = video(brain,
              label)
animation = camera.animate() # animation ready
HTML(animation.to_html5_video()) # displaying the animation

In [23]:
SHAPE = [241, 336, 283]

brain_mean = np.empty(SHAPE)
label_total = np.empty(SHAPE)

for pb,pl in tqdm(zip(paths_brains, paths_labels)):
    
    brain = np.load(pb)
    label = np.load(pl)
    
    brain_mean+=brain
    label_total+=label

80it [00:24,  3.23it/s]


In [24]:
brain_mean_ = brain_mean / brain_mean.max()
label_total_ = label_total / len(paths_labels) #label_total.sum()

In [25]:
def video(brain_tensor, mask_tensor=None, n_slides=100):

    '''
    brain_tensor - single ndarray brain [H,W,D]
    '''
    
    fig, ax = plt.subplots()
    X_max, Y_max, Z_max = brain_tensor.shape
    camera = Camera(fig)
    
    for y_slice_pos in np.linspace(0,Y_max-1, n_slides, dtype=int):
        
        brain_tensor_slice = brain_tensor[:,y_slice_pos,:]
        ax.imshow(brain_tensor_slice, 'gray')
        
        if mask_tensor is not None:
            mask_tensor_slice = mask_tensor[:,y_slice_pos,:]
            im = ax.imshow(mask_tensor_slice, 'jet', interpolation='none', alpha=0.7)
        
#         if y_slice_pos == 0:
#             plt.clim(0, 1)
#         matplotlib.colorbar.ColorbarBase(ax, cmap=matplotlib.cm.cool,
#                             orientation='vertical')
    
#         cbar = fig.colorbar(mp)
#         cbar.set_label('Relative abundance', rotation=270, fontsize=FONTSIZE, labelpad=15)
        
        camera.snap()
        
    
    return camera   

In [26]:
plt.ioff()
camera = video(brain_mean_,
              label_total_)
animation = camera.animate() # animation ready
HTML(animation.to_html5_video()) # displaying the animation

# Parcellation

In [13]:
%env FREESURFER_HOME=/media/tom/ibulygin/freesurfer

env: FREESURFER_HOME=/media/tom/ibulygin/freesurfer


In [15]:
!source $FREESURFER_HOME/SetUpFreeSurfer.sh

In [20]:
%env SUBJECTS_DIR=/media/tom/ibulygin/freesurfer_output

env: SUBJECTS_DIR=/media/tom/ibulygin/freesurfer_output


In [ ]:
my_subject=sample
my_NIfTI=/path/to/NIfTI.nii.gz
recon-all -i $my_NIfTI -s $my_subject -all